In [1]:
import subprocess
import shlex
import os
import re
import glob

import numpy as np

from subprocess import PIPE
from pathlib import Path

In [2]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [3]:
def submit_independent_spatially_explicit(
    delta_t, step_slice, dedup_cache, event_slice, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    if memory > 96:
        ncpus = max(int(np.ceil(memory / 120)), 1) * 10
        memory = ncpus * 12
    else:
        ncpus = 1
        
    max(int(np.ceil(memory / 24)), 1) * 24
    
    config = fr"""
    #PBS -lselect={1}:ncpus={ncpus}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent(
            delta_t: {delta_t},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: Monolithic(event_slice: {event_slice}),
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [4]:
def submit_cuda_spatially_explicit(
    delta_t, block_size, grid_size, step_slice, dedup_cache, event_slice, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=24,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    ngpus = max(int(np.ceil(memory / 24)), 1) * 24
    
    config = fr"""
    #PBS -lselect={1}:ncpus={4*ngpus}:mem={24*ngpus}gb:ngpus={ngpus}:gpu_type=P100
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: CUDA(
            device: {0},
            ptx_jit: {str(True).lower()},
            delta_t: {delta_t},
            block_size: {block_size},
            grid_size: {grid_size},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: Monolithic(event_slice: {event_slice}),
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [5]:
def submit_monolithic_spatially_explicit(
    algorithm,
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    if memory > 96:
        ncpus = max(int(np.ceil(memory / 120)), 1) * 10
        memory = ncpus * 12
    else:
        ncpus = 1
    
    config = fr"""
    #PBS -lselect={1}:ncpus={ncpus}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: {algorithm}(
            parallelism_mode: Monolithic,
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [6]:
for sample, memory, walltime in [
    (0.00000025, 16, 1), (0.0000025, 16, 2), (0.000025, 16, 4), (0.00025, 16, 12), (0.0025, 16, 24), (0.025, 32, 32),
]:
    submit_monolithic_spatially_explicit(
        "Classical",
        walltime=walltime, memory=memory, repeats=10, speciation=0.000001, sample=sample,
        output=f"classical/pbs.{sample}",
    )

for sample, memory, walltime in [
    (0.00000025, 16, 1), (0.0000025, 16, 6), (0.000025, 16, 12), (0.00025, 16, 24), (0.0025, 16, 32), (0.025, 32, 40),
]:
    submit_monolithic_spatially_explicit(
        "Gillespie",
        walltime=walltime, memory=memory, repeats=10, speciation=0.000001, sample=sample,
        output=f"gillespie/pbs.{sample}",
    )
    
for sample, memory, walltime in [
    (0.00000025, 16, 1), (0.0000025, 16, 1), (0.000025, 16, 1), (0.00025, 16, 1),
    (0.0025, 16, 2), (0.025, 32, 3), (0.25, 120, 16)#, (1.0, 480, 48)
]:
    submit_monolithic_spatially_explicit(
        "SkippingGillespie",
        walltime=walltime, memory=memory, repeats=10, speciation=0.000001, sample=sample,
        output=f"skipping-gillespie/pbs.{sample}",
    )

In [7]:
for sample, memory, event_slice, walltime in [
    (0.00000025, 16, 1000, 1), (0.0000025, 16, 10000, 1), (0.000025, 16, 100000, 2), (0.00025, 16, 1000000, 4),
    (0.0025, 16, 10000000, 4), (0.025, 16, 100000000, 6), (0.25, 96, 500000000, 12)#, (1.0, 480, 4000000000, 48)
]:
    submit_independent_spatially_explicit(
        2.0, 10, f"Relative(factor: {1.0})", event_slice, # delta_t, step_slice, dedup_cache, event_slice
        walltime=walltime, memory=memory, repeats=10, speciation=0.000001, sample=sample,
        output=f"independent/pbs.{sample}",
    )

In [8]:
# for sample, memory, event_slice in [
#     (0.00000025, 16, 1000), (0.0000025, 16, 10000), (0.000025, 16, 100000), (0.00025, 16, 1000000),
#     (0.0025, 16, 10000000), (0.025, 32, 100000000), (0.25, 120, 500000000)
# ]:
#     submit_cuda_spatially_explicit(
#         """delta_t, block_size, grid_size, step_slice, dedup_cache, event_slice,""" # sweetspot
#         walltime=24, memory=memory, repeats=1, speciation=0.000001, sample=sample,
#         output=f"cuda/pbs.{sample}",
#     )